In [62]:
import os
import multiprocessing as mp
import itertools
from time import sleep


input_directory = '/Users/isaccocenacchi/Desktop/Tirocinio/MAGs_short'
mags = [os.path.join(dirpath,filename) 
             for dirpath, _, filenames in os.walk(input_directory) 
             for filename in filenames 
             if filename.endswith('.bz2')
            ]
print("Mags: ", len(mags))
num_cpus = mp.cpu_count()
print("CPUs: ", num_cpus)


def batched_mags(iterable, n):
    iterator = iter(iterable)
    while (batch := tuple(itertools.islice(iterator, n))):
        yield batch

print(len(list(batched_mags(mags, num_cpus))))

-(-3//2)


Mags:  329
CPUs:  8
42


2

In [ ]:
from glob import glob
import os
import bz2
import argparse
import time
import subprocess
from concurrent.futures import ProcessPoolExecutor
from functools import partial
import matplotlib.pyplot as plt
import pandas as pd
import shutil
import multiprocessing as mp
import itertools


# run_minced.py
# Version 0.1
# 18/04/2024
# by isacchetto

input_directory = '/Users/isaccocenacchi/Desktop/Tirocinio/MAGs_short'
output_dir = "{}_crispr_{}".format(input_directory, time.strftime("%Y%m%d%H%M%S"))
measures = []

# mags = glob(os.path.join(input_directory, '**/*.bz2'), recursive=True)

# mags = []
# for root, _, files in os.walk(input_directory):
#         for filename in files:
#             if filename.endswith('.bz2'):
#                   mags.append(os.path.join(root, filename))
# print(len(mags))

mags = [os.path.join(dirpath,filename) 
             for dirpath, _, filenames in os.walk(input_directory) 
             for filename in filenames 
             if filename.endswith('.bz2')
            ]
print("Mags: ", len(mags))
num_cpus = mp.cpu_count() - 1
print("CPUs: ", num_cpus)


In [ ]:
def decompress1(mags, output_dir):
    for mag in mags:
        start_time = time.time()
        unzipped_mag=os.path.join(output_dir, os.path.relpath(mag[:-4], input_directory))
        os.makedirs(os.path.dirname(unzipped_mag), exist_ok=True)
        with open(unzipped_mag, 'wb') as new_file, bz2.BZ2File(mag, 'rb') as file:
            for data in iter(lambda : file.read(100 * 1024), b''):
                new_file.write(data)
        end_time = time.time()
        measures.append(end_time - start_time)
    mean_time = sum(measures) / len(measures)
    print("Tempo medio:", mean_time, "secondi")
    print("Tempo totale:", sum(measures), "secondi")

def batched_mags(iterable, n):
    it = iter(iterable)
    while (batch := tuple(itertools.islice(it, n))):
        yield batch

with mp.Pool(num_cpus) as pool:
        pool.map(decompress1, batched_mags(mags, num_cpus))

In [ ]:
times = pd.Series(measures)
times.head()

times.plot.line()
plt.xlabel('MAG')
plt.ylabel('Time (s)')
plt.title('Time to unzip MAGs')

plt.show()

In [ ]:
def decompress2(mags, output_dir):
    measures.clear()
    for mag in mags:
        start_time = time.time()
        unzipped_mag=os.path.join(output_dir, os.path.relpath(mag[:-4], input_directory))
        os.makedirs(os.path.dirname(unzipped_mag), exist_ok=True)
        with open(unzipped_mag, 'wb') as new_file, open(mag, 'rb') as file:
            decompressor = bz2.BZ2Decompressor()
            for data in iter(lambda : file.read(100 * 1024), b''):
                new_file.write(decompressor.decompress(data))
        end_time = time.time()
        measures.append(end_time - start_time)
    mean_time = sum(measures) / len(measures)
    print("Tempo medio:", mean_time, "secondi")
    print("Tempo totale:", sum(measures), "secondi")

decompress2(mags, "{}_crispr_{}".format(input_directory, time.strftime("%Y%m%d%H%M%S")))

In [ ]:
from multiprocessing import Pool
import subprocess

def run_script(input):
    (command,arg_str)=input
    print("Starting command :{} with argument {}".format(command, arg_str))
    result = subprocess.call(command+" "+arg_str, shell=True) 
    print("Completed command :{} with argument {}".format(command, arg_str))
    return result

if __name__ == '__main__':
    with Pool(5) as p: # choose appropriate level of parallelism
        # choose appropriate command and argument, can be fetched from sys.argv if needed
        exit_codes = p.map(run_script, [('echo','hello1'), ('echo','hello2')])
        print("Exit codes : {}".format(exit_codes))

In [ ]:
def decompress4(mags, output_dir):
    measures.clear()
    for mag in mags:
        start_time = time.time()
        unzipped_mag=os.path.join(output_dir, os.path.relpath(mag[:-4], input_directory))
        os.makedirs(os.path.dirname(unzipped_mag), exist_ok=True)
        sts=subprocess.Popen(['bunzip2', '-kc', mag, '>', unzipped_mag])
        sts.wait()
        # os.system('bunzip2 -k {}'.format(mag))
        end_time = time.time()
        measures.append(end_time - start_time)
    mean_time = sum(measures) / len(measures)
    print("Tempo medio:", mean_time, "secondi")
    print("Tempo totale:", sum(measures), "secondi")

decompress4(mags, "{}_crispr_{}".format(input_directory, time.strftime("%Y%m%d%H%M%S")))
